In [ ]:
import ptoclient
import json

In [ ]:
token = None
c = ptoclient.PTOClient("https://mami.cloudlab.zhaw.ch/mami/pto", token)

## 2014: path dependency only

In [ ]:
dates_2014 = ["140827", "140904", "140909"]
vp_2014 = ["lon", "nyc", "sgp"]

pathdep_sets_2014 = {}
for date in dates_2014:
    pathdep_sets_2014[date] = []

sets_ecn_2014 = c.sets_by_metadata(source="https://mami.cloudlab.zhaw.ch/mami/pto/raw/ecn-2014")
for set_url in sets_ecn_2014:
    m = c.retrieve_set(url=set_url).metadata()
    
    for date in dates_2014:
        if date in m["_sources"][0]:
            pathdep_sets_2014[date].append(m["__link"].split("/")[-1]) 

In [ ]:
commands_2014 = []
for date in pathdep_sets_2014:
    commands_2014.append("ptocat {} | ecn_pathdep > ecn_pathdep_{}.obs".format(" ".join(pathdep_sets_2014[date]), date))
commands_2014

In [ ]:
with open("pathdep-14.sh", mode="w") as sf:
    for line in commands_2014:
        sf.write(line+"\n")

## 2016: path dependency only

In [ ]:
dates_2016 = ["160719", "160727", "160804"]
vp_2016 = ["lon", "nyc", "sgp"]

pathdep_sets_2016 = {}
for date in dates_2016:
    pathdep_sets_2016[date] = []

sets_ecn_2016 = c.sets_by_metadata(source="https://mami.cloudlab.zhaw.ch/mami/pto/raw/ecn-2016")
for set_url in sets_ecn_2016:
    m = c.retrieve_set(url=set_url).metadata()
    
    for date in dates_2016:
        if date in m["_sources"][0]:
            pathdep_sets_2016[date].append(m["__link"].split("/")[-1]) 

In [ ]:
commands_2016 = []
for date in pathdep_sets_2016:
    commands_2016.append("ptocat {} | ecn_pathdep > ecn_pathdep_{}.obs".format(" ".join(pathdep_sets_2016[date]), date))
commands_2016

In [ ]:
with open("pathdep-16.sh", mode="w") as sf:
        sf.write(line+"\n")

## 2017-02: stabilization

In [ ]:
runs = {}
vps = {}

setids_2017_02 = {}

sets_ecn_2017_02 = c.sets_by_metadata(source="https://mami.cloudlab.zhaw.ch/raw/ecn-2017-02")
for set_url in sets_ecn_2017_02:
    m = c.retrieve_set(url=set_url).metadata()
    leaf = m["_sources"][0].split("/")[-1].split(".")[0]
    setid = m["__link"].split("/")[-1]
    (run, do, vp, sz, idx) = leaf.split("-")
    
    if run in runs:
        runs[run] += 1
    else:
        runs[run] = 1
    
    if vp in vps:
        vps[vp] += 1
    else:
        vps[vp] = 1
    
    if run not in setids_2017_02:
        setids_2017_02[run] = {}
    
    if vp not in setids_2017_02[run]:
        setids_2017_02[run][vp] = ["","",""]
    
    setids_2017_02[run][vp][int(idx)] = setid

del(runs['OBGtb2KR2t'])
del(runs['zQS99j6tAJ'])

In [ ]:
commands_2017_02_stable = []

for run in runs:
    for vp in vps:
        if vp in setids_2017_02[run]:
            commands_2017_02_stable.append("ptocat {} | ecn_stabilizer > ecn_stable_{}_{}.obs".format(
            " ".join(setids_2017_02[run][vp]), run, vp))

commands_2017_02_stable

In [ ]:
with open("stabilize-17.sh", mode="w") as sf:
    for line in commands_2017_02_stable:
        sf.write(line+"\n")

## 2017-02: path dependency

In [ ]:
setids_stable_2017_02 = {}

sets_ecn_stable = c.sets_by_metadata(condition="ecn.stable.connectivity.works")
for set_url in sets_ecn_stable:
    m = c.retrieve_set(url=set_url).metadata()
    sources = m["_sources"]
    setid = m["__link"].split("/")[-1]
            
    sm = c.retrieve_set(url=sources[0]).metadata()
    smleaf = sm["_sources"][0].split("/")[-1].split(".")[0]
    run = smleaf.split("-")[0]
    
    if run not in setids_stable_2017_02:
        setids_stable_2017_02[run] = []
    
    setids_stable_2017_02[run].append(setid)

In [ ]:
commands_2017_02_pathdep = []

for run in setids_stable_2017_02:
    commands_2017_02_pathdep.append("ptocat {} | ecn_pathdep > ecn_pathdep_{}.obs".format(
            " ".join(setids_stable_2017_02[run]), run))
commands_2017_02_pathdep

In [ ]:
with open("pathdep-17.sh", mode="w") as sf:
    for line in commands_2017_02_stable:
        sf.write(line+"\n")